In [1]:
import requests
import json
from base64 import b64encode
import tkinter as tk
import random
from bs4 import BeautifulSoup

In [2]:
client_id = "e291792abd4b4a1db646f561c823beb9"
client_secret = "26e6df1ae3614825940739dd03c31feb"

auth_url = 'https://accounts.spotify.com/api/token'
auth_header = 'Basic ' + b64encode((client_id + ':' + client_secret).encode()).decode()
url = "https://api.spotify.com/v1/search"

auth_data = {
    'grant_type': 'client_credentials'
}

auth_response = requests.post(auth_url, headers={'Authorization': auth_header}, data=auth_data)

if auth_response.status_code == 200:
    token = auth_response.json().get('access_token')
    print('Token:', token)
else:
    print('Error:', auth_response.status_code)
    print(auth_response.text)





Token: BQDMNPfEV2VymjqEpEMlkZ0yFaz_V5yQlX5u1i4G9TpmanxeChJ4brjq8UEx1qLw4zkKuhFKZ1E77Qh-uHkaXjExRG_fx4n-LtBTLxScG4BVBE1HL9B05h4crPO3TYlsyp5JtnvtHyU


In [3]:
token = "BQDLS51gIeE-tdAjx3S78yTN_8nus2WYiwXYS_tQE6ktSSIp2A-DaB8pwY-7Gm6iRnXOfkfGM3LmkB6LU-j07Vm3TRvAYmUUgVdAie09YVmUqwqrUtevOQAfvuBkF74RJQGnnECTNbs"

#artist_name = "Taylor Swift"

def get_spotify_id(artist_name):
    url = "https://api.spotify.com/v1/search"
    headers = {
        "Authorization": f"Bearer {token}"
    }
    params = {
        "q": artist_name,
        "type": "artist",
        "limit": 1  
    }

    response = requests.get(url, headers=headers, params=params)

    if response.status_code == 200:
        results = response.json()
        if results["artists"]["items"]:
            artist = results["artists"]["items"][0]
            id = artist["id"]
            return id
        else:
            return "No artist found."
    else:
        return f"Error: {response.status_code} - {response.text}"

In [4]:
API_KEY = "OdOTpaxGkQyamSaW2EkeanimV3vOKKdmyBTwQHGHxErCN97F3c1Z4kVR8UJWgM85"
HEADERS = {"Authorization": f"Bearer {API_KEY}"}

def get_lyrics(artist):
    response = requests.get("https://api.genius.com/search", headers=HEADERS, params={"q": artist})
    data = response.json()

    if not data["response"]["hits"]:
        print("No songs found.\n")
        return []
    else:
        song = random.choice(data["response"]["hits"])["result"]
        
        title = song["title"]
        
        hint = song["title"][0]

        page = requests.get(song["url"])
        soup = BeautifulSoup(page.text, "html.parser")

        lyrics_divs = soup.find_all("div", class_=lambda x: x and "Lyrics__Container" in x)

        full_lyrics = ""
        
        for i in lyrics_divs:
            full_lyrics += i.text.strip()

        words = full_lyrics.split() 

        if len(words) >= 10:
            stop = random.randint(10, len(words))
            start = stop - 10
            consecutive_words = words[start:stop]
            song_lst = [title, " ".join(consecutive_words), hint]
            return song_lst
                
        else:
            print("Not enough words in the lyrics.\n")
            return []

#added for reference
def q2():

    score = 0

    #question 1
    response_2 = input("Enter an artist's name: ")
    question_1 = get_lyrics(response_2)

    response_3 = input(f"Enter the song title for the following lyrics: {question_1[1]}\n")
    if response_3.lower() == question_1[0].lower():
        score += 10
        print(f"\nCorrect! Your score is {score}/30.\n")
    else:
        print(f"\nIncorrect. The correct answer was {question_1[0]}. Your current score is {score}/30\n")

    #question 2
    question_2 = get_lyrics(response_2)
    response_5 = input(f"Enter the song title for the following lyrics: {question_2[1]}\n")
    if response_5.lower() == question_2[0].lower():
        score += 10
        print(f"\nCorrect! Your score is {score}/30.\n")
    else:
        print(f"\nIncorrect. The correct answer was {question_2[0]}. Your current score is {score}/30\n")

    #question 3
    question_3 = get_lyrics(response_2)
    response_7 = input(f"Enter the song title for the following lyrics: {question_3[1]}\n")
    if response_7.lower() == question_3[0].lower():
        score += 10
        print(f"\nCorrect! Your score is {score}/30.\n")
    else:
        print(f"\nIncorrect. The correct answer was {question_3[0]}. Your current score is {score}/30\n")

    return f"Your score is {score}/30."

In [12]:
q2()


Correct! Your score is 10/30.


Correct! Your score is 20/30.


Incorrect. The correct answer was All Too Well (10 Minute Version) (Taylor’s Version) [Live Acoustic]. Your current score is 20/30



'Your score is 20/30.'

In [ ]:
root = tk.Tk()
root.geometry("610x760")

level2frame = tk.Frame(root, bg="#67D771")

API_KEY = "OdOTpaxGkQyamSaW2EkeanimV3vOKKdmyBTwQHGHxErCN97F3c1Z4kVR8UJWgM85"
HEADERS = {"Authorization": f"Bearer {API_KEY}"}

def get_lyrics_tk(artist):
    response = requests.get("https://api.genius.com/search", headers=HEADERS, params={"q": artist})
    data = response.json()

    if not data["response"]["hits"]:
        no_artist_label = tk.Label(level2frame, text="Artist not found. Please try again.")
        no_artist_label.pack()
        return None
    else:
        song = random.choice(data["response"]["hits"])["result"]
        
        title = song["title"]
        
        hint = song["title"][0]

        page = requests.get(song["url"])
        soup = BeautifulSoup(page.text, "html.parser")

        lyrics_divs = soup.find_all("div", class_=lambda x: x and "Lyrics__Container" in x)

        full_lyrics = ""
        
        for i in lyrics_divs:
            full_lyrics += i.text.strip()

        words = full_lyrics.split() 

        if len(words) >= 10:
            stop = random.randint(10, len(words))
            start = stop - 10
            consecutive_words = words[start:stop]
            song_lst = [title, " ".join(consecutive_words), hint]
            lyrics_label = tk.Label(level2frame, text=f"{song_lst[1]} Guess the song. Hint: {song_lst[2]}")
            lyrics_label.pack()
            return song_lst
                
        else:
            no_lyrics_label = tk.Label(level2frame, text="Not enough lyrics. Please try a different artist.")
            no_lyrics_label.pack()
            return None

def correct_lyrics(guess, correct_answer):
    if guess.lower() == correct_answer.lower():
        correct_label = tk.Label(level2frame, text="Correct.", font=("Helvetica", 14, "bold"), bg="#67D771")
        correct_label.pack()
    else:
        incorrect_label = tk.Label(level2frame, text="Incorrect.", font=("Helvetica", 14, "bold"), bg="#67D771")
        incorrect_label.pack()

def clear():
    question_label1.configure(text='')
    print('hi')

def forget(a, b, c, d,e):
    a.delete(0, "end")
    b.forget()
    c.forget()
    d.forget()
    e = True
    return e

def lyr_question(artist, lyrics, question_label, boo):
    song_string = tk.StringVar()
    song_entry = tk.Entry(level2frame, textvariable=song_string, font=("Helvetica", 14, "bold"), bg="#67D771")
    song_button = tk.Button(level2frame, text="Submit", font=("Helvetica", 14, "bold"), bg="white",
                        command=lambda: correct_lyrics(song_string.get(), lyrics[0]))
    next_q_button = tk.Button(level2frame, text="Next question", font=("Helvetica", 14, "bold"), bg="white",
                        command=lambda:[clear(),forget(song_entry, song_button,next_q_button ,question_label, boo)])
    song_entry.pack()
    song_button.pack()
    next_q_button.pack()

def level2game(artist):
    #q1
    q1_boo=False
    question_label1 = tk.Label(level2frame, text="Question 1/5. Guess the song:")
    lyr1 = get_lyrics_tk(artist)
    if lyr1 is not None:
        lyr_question(artist, lyr1, question_label1, q1_boo)
        #q2
        q2_boo=False
        if q1_boo == True:
            lyr2 = get_lyrics_tk(artist)
            if lyr2 is not None:
                question_label2 = tk.Label(level2frame, text="Question 2/5. Guess the song:")
                if lyr2 == lyr1:
                    lyr2 = get_lyrics_tk(artist)
                lyr_question(artist, lyr2, question_label2, q2_boo)
                #q3
                q3_boo = False
                if q2_boo == True:
                    lyr3 = get_lyrics_tk(artist)
                    if lyr3 is not None:
                        question_label3 = tk.Label(level2frame, text="Question 3/5. Guess the song:")
                        if lyr3 == lyr1 or lyr3 == lyr2:
                            lyr3 = get_lyrics_tk(artist)
                        lyr_question(artist, lyr3, question_label3, q3_boo)
                        #q4
                        q4_boo = False
                        if q3_boo == True:
                            lyr4 = get_lyrics_tk(artist)
                            if lyr4 is not None:
                                question_label4 = tk.Label(level2frame, text="Question 4/5. Guess the song:")
                                if lyr4 == lyr1 or lyr4 == lyr2 or lyr4 == lyr3:
                                    lyr4 = get_lyrics_tk(artist)
                                lyr_question(artist, lyr4, question_label4, q4_boo)
                                #q5
                                q5_boo = False
                                if q4_boo == True:
                                    lyr5 = get_lyrics_tk(artist)
                                    if lyr5 is not None:
                                        question_label5 = tk.Label(level2frame, text="Question 5/5. Guess the song:")
                                        if lyr5 == lyr1 or lyr5 == lyr2 or lyr5 == lyr3 or lyr5 == lyr4:
                                            lyr5 = get_lyrics_tk(artist)
                                        lyr_question(artist, lyr5, question_label5, q5_boo)


            
artist_string = tk.StringVar()
artist_label = tk.Label(level2frame, text="Enter an artist's name to guess a song based on lyrics.", font=("Helvetica", 14, "bold"), bg="#67D771")
artist_input = tk.Entry(level2frame, textvariable=artist_string, font=("Helvetica", 14, "bold"), bg="#67D771")
artist_button = tk.Button(level2frame, text="Submit", font=("Helvetica", 14, "bold"), bg="white",
                          command=lambda: level2game(artist_string.get()))

level2frame.pack()
artist_label.pack()
artist_input.pack()
artist_button.pack()

root.mainloop()

hi
